In [2]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100).view(1, -1, 1).repeat(4, 1, 1)
# True functions are sin(2pi x), cos(2pi x), sin(pi x), cos(pi x)
sin_y = torch.sin(train_x[0] * (2 * math.pi)) + 0.5 * torch.rand(1, 100, 1)
sin_y_short = torch.sin(train_x[0] * (math.pi)) + 0.5 * torch.rand(1, 100, 1)
cos_y = torch.cos(train_x[0] * (2 * math.pi)) + 0.5 * torch.rand(1, 100, 1)
cos_y_short = torch.cos(train_x[0] * (math.pi)) + 0.5 * torch.rand(1, 100, 1)
train_y = torch.cat((sin_y, sin_y_short, cos_y, cos_y_short)).squeeze(-1)

In [9]:
train_x.size()

torch.Size([4, 100, 1])

In [4]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([4]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(batch_shape=torch.Size([4])),
            batch_shape=torch.Size([4])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood(batch_shape=torch.Size([4]))
model = ExactGPModel(train_x, train_y, likelihood)

In [16]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    print(train_y.size())
    # Calc loss and backprop gradients
    loss = -mll(output, train_y).sum()
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))
    optimizer.step()

torch.Size([4, 100])
Iter 1/50 - Loss: 2.791
torch.Size([4, 100])
Iter 2/50 - Loss: 2.633
torch.Size([4, 100])
Iter 3/50 - Loss: 2.477
torch.Size([4, 100])
Iter 4/50 - Loss: 2.325
torch.Size([4, 100])
Iter 5/50 - Loss: 2.174
torch.Size([4, 100])
Iter 6/50 - Loss: 2.025
torch.Size([4, 100])
Iter 7/50 - Loss: 1.875
torch.Size([4, 100])
Iter 8/50 - Loss: 1.724
torch.Size([4, 100])
Iter 9/50 - Loss: 1.571
torch.Size([4, 100])
Iter 10/50 - Loss: 1.417
torch.Size([4, 100])
Iter 11/50 - Loss: 1.261
torch.Size([4, 100])
Iter 12/50 - Loss: 1.106
torch.Size([4, 100])
Iter 13/50 - Loss: 0.950
torch.Size([4, 100])
Iter 14/50 - Loss: 0.796
torch.Size([4, 100])
Iter 15/50 - Loss: 0.644
torch.Size([4, 100])
Iter 16/50 - Loss: 0.493
torch.Size([4, 100])
Iter 17/50 - Loss: 0.346
torch.Size([4, 100])
Iter 18/50 - Loss: 0.201
torch.Size([4, 100])
Iter 19/50 - Loss: 0.060
torch.Size([4, 100])
Iter 20/50 - Loss: -0.079
torch.Size([4, 100])
Iter 21/50 - Loss: -0.214
torch.Size([4, 100])
Iter 22/50 - Loss: -

In [15]:
train_y.s

torch.Size([4, 100])